# 使用混合式编程

 * 我们之前学习了如何使用Sequential类来串联多个层。为了使用混合式编程，下面我们将Sequential类替换成HybridSequential类

In [1]:
import sys
sys.path.append('../')

In [2]:
import mxnet as mx

C:\Users\lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from mxnet import nd,sym
from mxnet.gluon import nn
import time

In [4]:
def get_net():
    net = nn.HybridSequential()  # 这里使用 HybridSequential 类。
    net.add(nn.Dense(256, activation='relu'),
            nn.Dense(128, activation='relu'),
            nn.Dense(2))
    net.initialize(ctx= mx.gpu())
    return net

In [5]:
x = nd.random.uniform(shape = (1,512),ctx= mx.gpu())
net = get_net()
print(net(x))



[[0.05828758 0.0332496 ]]
<NDArray 1x2 @gpu(0)>


In [6]:
net.hybridize()
net(x)


[[0.05828758 0.0332496 ]]
<NDArray 1x2 @gpu(0)>

只有继承 HybridBlock 类的层才会被优化计算。例如， HybridSequential 类和Gluon 提供的 Dense 类都是 HybridBlock 类的⼦类，它们都会被优化计算。如果⼀个层只是继承⾃ Block 类而不是 HybridBlock 类，那么它将不会被优化。

In [9]:
def benchmark(net, x):
    start = time.time()
    for i in range(10000):
        _ = net(x)
    nd.waitall() # 等待所有计算完成⽅便计时。
    return time.time() - start

In [10]:
net = get_net()
print('before hybridizing: %.4f sec' % (benchmark(net, x)))
net.hybridize()
print('after hybridizing: %.4f sec' % (benchmark(net, x)))

before hybridizing: 2.6369 sec
after hybridizing: 2.0844 sec


In [11]:
net.export('my_mlp')

In [12]:
x = sym.var('data')
net(x)

<Symbol dense8_fwd>

## 使用HybridBlock类构造模型

In [28]:
class HybridNet(nn.HybridBlock):
    def __init__(self,**kwargs):
        super(HybridNet,self).__init__(**kwargs)
        self.hidden = nn.Dense(10)
        self.output = nn.Dense(2)
    
    def hybrid_forward(self,F,x):
        #x.asnumpy()
        print('F: ', F)
        print('x: ', x)
        x = F.relu(self.hidden(x))
        print('hidden: ', x)
        return self.output(x)


In [29]:
net = HybridNet()

In [30]:
net.initialize(ctx= mx.gpu())

In [31]:
x = nd.random.normal(shape=(1, 4),ctx=mx.gpu())
net(x)

F:  <module 'mxnet.ndarray' from 'C:\\Users\\lenovo\\Anaconda3\\lib\\site-packages\\mxnet\\ndarray\\__init__.py'>
x:  
[[-0.9246624   0.9911375  -0.4820686  -0.61760885]]
<NDArray 1x4 @gpu(0)>
hidden:  
[[0.02223664 0.02621048 0.         0.         0.         0.
  0.09675613 0.         0.         0.00331112]]
<NDArray 1x10 @gpu(0)>



[[ 0.00382923 -0.00562732]]
<NDArray 1x2 @gpu(0)>

In [32]:
net.hybridize()

In [33]:
net(x)

F:  <module 'mxnet.symbol' from 'C:\\Users\\lenovo\\Anaconda3\\lib\\site-packages\\mxnet\\symbol\\__init__.py'>
x:  <Symbol data>
hidden:  <Symbol hybridnet2_relu0>



[[ 0.00382923 -0.00562732]]
<NDArray 1x2 @gpu(0)>

In [34]:
net(x)


[[ 0.00382923 -0.00562732]]
<NDArray 1x2 @gpu(0)>